<a href="https://colab.research.google.com/github/andrybrew/socialmediaanalytic/blob/master/004_collecting_twitter_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Collecting Twitter Data

To collect twitter data in python, we can use Tweepy. Tweepy is the most popular Python Package for accessing the Twitter API, You can read the full documentation [HERE](https://tweepy.readthedocs.io/en/latest/). In this practice, we will practice to get tweet data using a specific keyword and save it as .CSV files

**Install & Import Libraries**

In [0]:
# Install Library
!pip install tweepy

In [0]:
# Import Libraries
import tweepy
import pandas as pd
import numpy as np
import sys
import csv

**Set API Key**

In [0]:
# Fill the API Key
consumer_key = '8c6g53lSeupE2anReETTs2vys'
consumer_secret = 'OltZDimReEajo9AEBAsAPqf78yyNzfrmH85aJzUzpqjAsH4Nxf'
access_token = '201094280-dsxUFTbrXldpm8YjkziHqkmFqCkF9aRH4lLvq1oE'
access_token_secret = 'VBpKAHnNCQjGjRpCLsrHROHufWX8tEbP3EAYblbYKNv5c'

# Auth. 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
api

## a.Crawling

Collect the past tweets.

### By Keyword

**Set Tweet Requirements**

In [0]:
# Set Word to Follow
keyword = 'corona'

# Set Number of Tweets
number_of_tweets = 100

# Tweet Mode
tweet_mode = 'extended'

# Set Language
language = 'en'

**Get the Tweets**

In [0]:
# Finding Tweets by Keyword
crawling_keyword = api.search(keyword, count=number_of_tweets, tweet_mode = tweet_mode, lang=language)

# Show Tweets
crawling_keyword

In [0]:
# Create Tweets Data Frame
df_crawling_keyword = pd.DataFrame({
                      'time' : [tweet.created_at for tweet in crawling_keyword],
                      'description' : [tweet.user.description for tweet in crawling_keyword],
                      'usertweets' : [tweet.user.statuses_count for tweet in crawling_keyword],
                      'source' : [tweet.user.screen_name for tweet in crawling_keyword],
                      'target' : [tweet.in_reply_to_screen_name for tweet in crawling_keyword],
                      'verified' : [tweet.user.verified for tweet in crawling_keyword],
                      'text' : [tweet.full_text for tweet in crawling_keyword],
                      'hashtags' : [tweet.entities['hashtags'] for tweet in crawling_keyword],
                      'location' : [tweet.user.location for tweet in crawling_keyword],
                      'following' : [tweet.user.friends_count for tweet in crawling_keyword],
                      'followers' : [tweet.user.followers_count for tweet in crawling_keyword],
                      'retweets' : [tweet.retweet_count for tweet in crawling_keyword],
                      })

df_crawling_keyword

**Save Tweets**

In [0]:
# Save as .CSV
df_crawling_keyword.to_csv('crawling_keyword.csv')

### User Timeline

**Set Tweets Requirement**

In [0]:
# User to Follow
user = '@raisa9960'

# Set Number of Tweets
number_of_tweets = 200

# Tweet Mode
tweet_mode = 'extended'

In [0]:
# Finding Tweets by Username
crawling_timeline = api.user_timeline(screen_name = user,  count = number_of_tweets, tweet_mode = tweet_mode)

# Show Tweets
crawling_timeline

In [0]:
# Create Tweets Data Frame
df_crawling_timeline = pd.DataFrame({
                      'time' : [tweet.created_at for tweet in crawling_timeline],
                      'text' : [tweet.full_text for tweet in crawling_timeline],
                      'location' : [tweet.user.location for tweet in crawling_timeline],
                      })

df_crawling_timeline

## b.Streaming

Stream the real-time Tweets

**Create Streaming Function**

In [0]:
# Import Modules
from tweepy import Stream
from tweepy.streaming import StreamListener

### By Keyword

**Set Tweet Requirements**

In [0]:
# Set Word to Follow
keyword = 'corona'

**Stream the Tweets**

In [0]:
# Create an Object Called 'customStreamListener' with the Function Below

class CustomStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        print (status.created_at, status.user.description, status.user.statuses_count,
               status.user.screen_name, status.in_reply_to_screen_name, status.user.verified,
               status.text, status.entities['hashtags'], status.user.location, 
               status.user.friends_count, status.user.followers_count, status.retweet_count)
        
        # Writing status data
        with open('streaming_keyword.txt', 'a') as f:
            writer = csv.writer(f)
            writer.writerow([status.created_at, status.user.description, status.user.statuses_count, 
                             status.user.screen_name, status.in_reply_to_screen_name, status.user.verified,
                             status.text, status.entities['hashtags'], status.user.location, 
                             status.user.friends_count, status.user.followers_count, status.retweet_count])

    def on_error(self, status_code):
        print >> sys.stderr, 'Encountered error with status code:', status_code
        return True # Don't kill the stream

    def on_timeout(self):
        print >> sys.stderr, 'Timeout...'
        return True # Don't kill the stream

In [0]:
# Writing csv Titles
with open('streaming_keyword.txt', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['time', 'description', 'usertweets', 'source', 'target', 'verified',
                     'text', 'hashtag', 'location', 'following', 'followers', 'retweets'])

streamingAPI = tweepy.streaming.Stream(auth, CustomStreamListener())
streamingAPI.filter(track=keyword)

In [0]:
# Stop Streaming
def disconnect(self):
    if self.running is False:
        return
    self.running = False

In [0]:
# Show Streaming Result
df_streaming = pd.read_csv('streaming_keyword.txt', sep=",")
df_streaming